In [1]:
#!pip install py7zr 
import geopandas as gpd
import pandas as pd
import requests
from bs4 import BeautifulSoup
import zipfile
import py7zr
import os
import shutil
import getpass

#### Gestion du proxy

In [4]:
http_proxy = getpass.getpass(prompt='http_proxy')

http_proxy ········


In [3]:
https_proxy = getpass.getpass(prompt='https_proxy')

https_proxy ········


In [5]:
req = "https://geoservices.ign.fr/ocsge"

In [6]:
proxies = { "http"  : http_proxy, "https" : https_proxy}

In [7]:
page = requests.get(req, proxies=proxies)

In [8]:
soup = BeautifulSoup(page.text, 'html.parser')

In [9]:
downloads = soup.findAll('a', href=True)

Liste des départements disponibles

In [10]:
liste_ocsge = [download for download in downloads if "OCS-GE_2-0" in download['href']]
liste_ocsge = pd.DataFrame({ 'name' : [item.find('strong').text if item.find('strong') else None for item in liste_ocsge],
               'link' : [item['href'] for item in liste_ocsge] })


liste_ocsge['codgeo'] = liste_ocsge.name.str.split(" - ",expand=True)[0].str.slice(12,).str.strip()
liste_ocsge['year'] = liste_ocsge.name.str.extract(r'(\b\d{4}\b)')
temp = liste_ocsge.name.str.extract(r'(\b\d{4}-\d{4}\b)')
liste_ocsge.loc[~temp.isna().any(axis=1),'year'] = temp[~temp.isna().any(axis=1)][0]
liste_ocsge = liste_ocsge[['codgeo', 'year', 'link', 'name']]


In [11]:
liste_ocsge.head()

,codgeo,year,link,name
0,01,2021,https://data.geopf.fr/telechargement/download/...,Département 01 - Ain - 2021
1,01,2018,https://data.geopf.fr/telechargement/download/...,Département 01 - Ain - 2018
2,01,2018-2021,https://data.geopf.fr/telechargement/download/...,Département 01 - Ain - différentiel - 2018-2021
3,15,2022,https://data.geopf.fr/telechargement/download/...,Département 15 - Cantal - 2022
4,15,2020,https://data.geopf.fr/telechargement/download/...,Département 15 - Cantal - 2020


In [12]:
#liste_ocsge.to_json('dep_dispo_2025-01-21.json')

### Ouverture de la liste et traitement

In [55]:
liste_ocsge = pd.read_json('dep_dispo_2025-01-21.json')
liste_ocsge.shape

(198, 4)

In [56]:
liste_ocsge_dep = liste_ocsge.copy()
liste_ocsge_dep = liste_ocsge_dep[liste_ocsge_dep.year.str.len()==4]
liste_ocsge_dep = liste_ocsge_dep
liste_ocsge_dep.head()

,codgeo,year,link,name
0,01,2021,https://data.geopf.fr/telechargement/download/...,Département 01 - Ain - 2021
1,01,2018,https://data.geopf.fr/telechargement/download/...,Département 01 - Ain - 2018
3,15,2022,https://data.geopf.fr/telechargement/download/...,Département 15 - Cantal - 2022
4,15,2020,https://data.geopf.fr/telechargement/download/...,Département 15 - Cantal - 2020
6,38,2021,https://data.geopf.fr/telechargement/download/...,Département 38 - Isère - 2021


#### traitement spécifique

Je récupère les fichiers non téléchargés

In [49]:
import glob
liste_termine = glob.glob("files_ocsge/*")
liste_termine = [file[12:].split('.')[0] for file in liste_termine]

In [57]:
liste_ocsge_dep['name'] = liste_ocsge_dep.link.map(lambda x :x.split('/')[-1].split('.')[0])
liste_ocsge_dep = liste_ocsge_dep[~liste_ocsge_dep.name.isin(liste_termine)]

In [58]:
liste_ocsge_dep

,codgeo,year,link,name
3,15,2022,https://data.geopf.fr/telechargement/download/...,OCS-GE_2-0__SHP_LAMB93_D015_2022-01-01
4,15,2020,https://data.geopf.fr/telechargement/download/...,OCS-GE_2-0__SHP_LAMB93_D015_2020-01-01
12,43,2022,https://data.geopf.fr/telechargement/download/...,OCS-GE_2-0__SHP_LAMB93_D043_2022-01-01
13,43,2019,https://data.geopf.fr/telechargement/download/...,OCS-GE_2-0__SHP_LAMB93_D043_2019-01-01


Téléchargement

In [59]:
for url in liste_ocsge_dep.link:
  with requests.get(url, stream=True, proxies=proxies) as response:
    if response.status_code == 200:
        with open(url.rsplit('/', 1)[1], 'wb') as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)

  search_string = "OCCUPATION_SOL"
  with py7zr.SevenZipFile(url.rsplit('/', 1)[1], mode='r') as archive:
        # Get the list of files in the archive
        all_files = archive.getnames()

        # Filter files that contain the search string
        filtered_files = [f for f in all_files if search_string in f]

        # Extract only the filtered files
        if filtered_files:
          archive.extract(targets=filtered_files, path='ocsge_temp',recursive=False)

        for file in filtered_files:
                source_file = os.path.join('ocsge_temp', file)
                destination_file = os.path.join('ocsge', os.path.basename(file))
                os.makedirs(os.path.dirname(destination_file), exist_ok=True)
                if os.path.exists(source_file):  # Only move if the file was extracted
                    os.rename(source_file, destination_file)

  ocsge = gpd.read_file('ocsge/OCCUPATION_SOL.shp')
  ocsge.to_parquet('files_ocsge/' + url.rsplit('/', 1)[1].rsplit('.', 1)[0]+".parquet")

  shutil.rmtree('ocsge_temp')
  shutil.rmtree('ocsge')
  os.remove(url.rsplit('/', 1)[1])